## 固有表現認識

### 固有表現認識とは？

In [3]:
# !pip install spacy-alignments seqeval

#### データセットのダウンロード

In [4]:
from datasets import load_dataset

# データセットを読み込む
dataset = load_dataset("llm-book/ner-wikipedia-dataset")

Using custom data configuration default


Dataset ner-wikipedia-dataset downloaded and prepared to /root/.cache/huggingface/datasets/llm-book___ner-wikipedia-dataset/default/0.0.0/184bcf9be66116e777f2f534436226d47348676c93ba20cca58933f1b2b3b782. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 682.30it/s]


In [5]:
# データセットの形式と事例数を確認する
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 4274
    })
    validation: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 534
    })
    test: Dataset({
        features: ['curid', 'text', 'entities'],
        num_rows: 535
    })
})


In [6]:
from pprint import pprint

# 訓練セットの最初の二つの事例を表示する
pprint(list(dataset["train"])[:2])

[{'curid': '3638038',
  'entities': [{'name': 'さくら学院', 'span': [0, 5], 'type': 'その他の組織名'},
               {'name': 'Ciao Smiles', 'span': [6, 17], 'type': 'その他の組織名'}],
  'text': 'さくら学院、Ciao Smilesのメンバー。'},
 {'curid': '1729527',
  'entities': [{'name': 'レクレアティーボ・ウェルバ', 'span': [17, 30], 'type': 'その他の組織名'},
               {'name': 'プリメーラ・ディビシオン', 'span': [32, 44], 'type': 'その他の組織名'}],
  'text': '2008年10月5日、アウェーでのレクレアティーボ・ウェルバ戦でプリメーラ・ディビシオンでの初得点を決めた。'}]


#### データセットの分析

In [8]:
from collections import Counter
import pandas as pd
from datasets import Dataset

def count_label_occurrences(dataset: Dataset) -> dict[str, int]:
    """固有表現タイプの出現回数をカウント"""
    # 各事例から固有表現タイプを抽出したlistを作成する
    entities = [
        e["type"] for data in dataset for e in data["entities"]
    ]
    
    # ラベルの表現回数が多い順に並べる
    label_counts = dict(Counter(entities).most_common())
    return label_counts
    
label_counts_dict = {}
for split in dataset: # 各分割セットを処理する
    label_counts_dict[split] = count_label_occurrences(dataset[split])
# DataFrame形式で表示する
df = pd.DataFrame(label_counts_dict)
df.loc["合計"] = df.sum()
display(df)

,train,validation,test
人名,2394,299,287
法人名,2006,231,248
地名,1769,184,204
政治的組織名,953,121,106
製品名,934,123,158
施設名,868,103,137
その他の組織名,852,99,100
イベント名,831,85,93
合計,10607,1245,1333


#### スパンの重なる固有表現の存在を判定

In [10]:
def has_overlap(spans: list[tuple[int, int]]) -> int:
    """スパンの重なる固有表現の存在を判定"""
    sorted_spans = sorted(spans, key=lambda x: x[0])
    for i in range(1, len(sorted_spans)):
        # 前のスパンの終了位置が現在のスパンの開始位置より大きい場合、
        # 重なっているとする
        if sorted_spans[i - 1][1] > sorted_spans[i][0]:
            return 1
    return 0
    
# 各分割セットでスパンの重なる固有表現がある事例数を数える
overlap_count = 0
for split in dataset: # 各分割セットを処理する
    for data in dataset[split]: # 各事例を処理する
        if data["entities"]: # 固有表現の存在しない事例はスキップ
            # スパンのみのlistを作成する
            spans = [e["span"] for e in data["entities"]]
            overlap_count += has_overlap(spans)
    print(f"{split}におけるスパンが重複する事例数:{overlap_count}")

trainにおけるスパンが重複する事例数:0
validationにおけるスパンが重複する事例数:0
testにおけるスパンが重複する事例数:0
